In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as ss
import openpyxl

# S-factor calculation

Energy data from DECDATA, not from Firestone.
Separated into IC-electrons, beta-minus-electrons and auger-electrons.
Using Hough-UF-model for the male skeleton first, redo and re-check calculations.

We do have the specific energy absorbtion fractions for the lumbar vertebraes in a tabular format.

27/7/2020

Better still, make it general so we can have multiple skeletal sites and nuclides, to better get a grasp of the (full) model.

Need to convert some values from the PDF, direct import in excel returns only text-strings. This can be done better

Finnished now, have to re-calculate everything but finally we have something usefull. Set up the new analysis and the leave-1-out and all of the other stuff...

In [60]:
# Conversion of SAF-data from strings2nums:

wb = openpyxl.load_workbook(filename="utils/lumbar_saf_values.xlsx")

#print(dir(wb))
print(wb.sheetnames)
ws = (wb["Ark1"])

A = np.array([[i.value for i in j] for j in ws['A2':'L34']])
foo = ((A[4,4]))

fib = foo.astype(np.float)

print('6.57e-03')
print(foo)

['Tissue-AM-Masses', 'CranofocalBones', 'Mandibles', 'ProximalFemura', 'ProximalHumeri', 'OsCoxae', 'Sacrum', 'Clavicles', 'LumbarVertebrae', 'ThoracaicVertebrae', 'CervicalVertebrae', 'Ribs', 'Sternum', 'Scapulae', 'Ark1']
6.57e-03
0.00657


In [250]:
VERBOSE = True

saf_input = "utils/saf_values_male_redone.xlsx"
#saf_input = "utils/saf_values_female_redone.xlsx"

main_wb = openpyxl.load_workbook(filename=saf_input)
sheet_names = main_wb.sheetnames

# Remove unwanted sheets
non_organ_sheets = ["Tissue-AM-Masses", "Ark1"]
organ_list = sheet_names

for s in non_organ_sheets:
    if s in organ_list: organ_list.remove(s)

mass_data = pd.read_excel(saf_input, sheet_name="Tissue-AM-Masses")

list_of_sites_in_mass_file = (mass_data["SiteName"].values)

if VERBOSE:

    for organ in organ_list:
        if organ in list_of_sites_in_mass_file:
            print("Organ " + organ + " is in list")
        else:
            print("Warning, names might mis-match " + organ)
           
        
for organ in organ_list[7:8]:
    
    if VERBOSE:
        print(organ)
    
    # Get the mass of the organ with 100 % cellularity
    mass_value = mass_data[mass_data["SiteName"] == organ]["AM-in-MST-100-CF"].values[0]
    
    # Now grab the desired SAF-sheet:
    
    organ_saf_values = pd.read_excel(saf_input, sheet_name=organ)
        
    energy_look_up_vector = organ_saf_values["Energy"].values
    
    # Now grab the energy-spectrum
    
    energies = pd.read_excel("utils/decdataenergies.xlsx", sheet_name="AllElectrons")

    len_of_table = len(energies)
    
    energy_absorbtion_frame = pd.DataFrame(columns=["Energy", "Y", "phi"], 
                                       index=np.arange(len_of_table))

    energy_absorbtion_frame["Energy"] = energies["E (MeV)"]
    energy_absorbtion_frame["Y"] = energies["Y (/nt)"]
    
    cf = 0.1
    CF_string = ("CF" + str(int(cf*100)))
    
    for i in range(len_of_table):

        energy_to_look_up = (energy_absorbtion_frame.at[i, "Energy"])

        index = (np.abs(energy_look_up_vector-energy_to_look_up)).argmin()

        phi = (organ_saf_values.at[index, CF_string])*mass_value*cf
        
        energy_absorbtion_frame.at[i, "phi"] = phi
    
    print(energy_absorbtion_frame)
    
    Energy_per_desintegration = (np.sum(energy_absorbtion_frame["Energy"]*energy_absorbtion_frame["Y"]*energy_absorbtion_frame["phi"]))
    


Organ CranofocalBones is in list
Organ Mandibles is in list
Organ ProximalFemura is in list
Organ ProximalHumeri is in list
Organ OsCoxae is in list
Organ Sacrum is in list
Organ Clavicles is in list
Organ LumbarVertebrae is in list
Organ ThoracaicVertebrae is in list
Organ CervicalVertebrae is in list
Organ Ribs is in list
Organ Sternum is in list
Organ Scapulae is in list
LumbarVertebrae
      Energy             Y       phi
0   0.047527  1.220570e-01  0.594275
1   0.078445  5.302490e-04  0.306862
2   0.111507  9.104280e-02  0.220422
3   0.149135  7.863700e-01  0.149973
4   0.006164  1.097530e-03  0.978933
5   0.047468  5.170720e-02  0.594275
6   0.060401  1.188010e-04  0.484064
7   0.060883  4.135440e-05  0.484064
8   0.062088  5.099510e-05  0.484064
9   0.069547  4.845200e-05  0.484064
10  0.071243  2.680200e-04  0.306862
11  0.071646  1.344860e-05  0.306862
12  0.101705  5.528020e-03  0.220422
13  0.102187  3.474040e-02  0.220422
14  0.103392  3.067770e-02  0.220422
15  0.110851  1

In [254]:
# Lets test with the lymphoma-data:

test_data_male = pd.read_excel("test_of_absorbed_dose.xlsx", sheet_name="lumbarDataBetalutin")

test_data_male["AbsDoseTest"] = np.zeros(len(test_data))


for i in range(len(test_data)):
    Cum = (test_data_male["MBqHr"][i])
    mass = (test_data_male["Mass"][i])
    
    Volume = mass*1e-3
    mass = cf*1.03*Volume
    
    print("Cumulative over mass: " + str(1e-5*Cum/mass))
    
    test_data_male["AbsDoseTest"][i] = (Cum*Energy_per_desintegration*1.602e-13*1e6*3600/mass)
    
print(test_data_male)
    
print(ss.pearsonr(test_data_male["AbsDoseTest"], test_data_male["PBN Neut"]))
print(ss.shapiro(test_data_male["PBN"]))


Cumulative over mass: 2.4298744920113524
Cumulative over mass: 1.011448531872813
Cumulative over mass: 1.3497003639065692
Cumulative over mass: 1.759724000955927
Cumulative over mass: 1.94947178235099
Cumulative over mass: 3.8566068432478313
Cumulative over mass: 2.821817994060439
Cumulative over mass: 2.686973743186038
Cumulative over mass: 3.1152720572384
Cumulative over mass: 1.6384410523741992
Cumulative over mass: 2.486055082710475
Cumulative over mass: 1.0526983597455501
Cumulative over mass: 2.7007991036740444
Cumulative over mass: 2.60542645745471
Cumulative over mass: 1.7852768352090542
Cumulative over mass: 2.4449216094682407
Cumulative over mass: 3.5450823943591585
          MBqHr        Mass        PBN   PBN Neut Sex  AbsDoseTest
0   1244.878159   49.740000  11.382114  15.686275   F     3.947452
1    812.285213   77.970000  40.723982  40.000000   M     1.643148
2    762.123639   54.821491  19.607843  38.571429   M     2.192655
3   1268.761005   70.000000  31.141869  26.5625

C:\Users\Johan\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [232]:
# Lets test with the lymphoma-data:

test_data_female = pd.read_excel("test_of_absorbed_dose.xlsx", sheet_name="lumbarDataBetalutin")

test_data_female["AbsDoseTest"] = np.zeros(len(test_data))


for i in range(len(test_data)):
    Cum = (test_data_female["MBqHr"][i])
    mass = (test_data_female["Mass"][i])
    
    Volume = mass*1e-3
    Cum
    mass = cf*1.03*Volume
    
    test_data_female["AbsDoseTest"][i] = (Cum*Energy_per_desintegration*1.602e-13*1e6*3600/mass)
    
print(test_data_female)
    
print(ss.pearsonr(test_data_female["AbsDoseTest"], test_data_female["PBN Neut"]))

          MBqHr        Mass        PBN   PBN Neut Sex  AbsDoseTest
0   1244.878159   49.740000  11.382114  15.686275   F     2.027505
1    812.285213   77.970000  40.723982  40.000000   M     0.843960
2    762.123639   54.821491  19.607843  38.571429   M     1.126200
3   1268.761005   70.000000  31.141869  26.562500   M     1.468327
4    922.468952   45.940697  31.983806  43.478261   F     1.626654
5   2682.878504   67.539589   3.921569   1.200000   M     3.217981
6   1433.589559   49.324036   8.695652  11.904762   F     2.354546
7   2778.150999  100.381851  37.719298  28.000000   M     2.242031
8   1076.224512   33.540511   9.693878  17.647059   F     2.599406
9   1017.625260   60.300350  12.371134  13.953488   F     1.367127
10  2220.559492   86.719036  20.647773   9.876543   M     2.074383
11   825.667711   76.148987  46.907216  42.105263   M     0.878379
12   963.698540   34.642697  24.899598  27.272727   F     2.253567
13  1391.411185   51.848888  24.509804  25.641026   M     2.17

C:\Users\Johan\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [235]:
#print(test_data_male)
#print(test_data_female)

foobar = pd.DataFrame()
foobar["PBN"] = test_data_female["PBN"]
foobar["PBN Neut"] = test_data_female["PBN Neut"]

print(test_data_male)
print(test_data_female)

foobar["Dose"] = np.zeros(len(foobar))


for i in range(len(foobar)):
    if (test_data_male["Sex"][i]) == "F":
        foobar["Dose"][i] = test_data_female["AbsDoseTest"][i]
        
    else:
        print("Itsa a boy!")
        foobar["Dose"][i] = test_data_male["AbsDoseTest"][i]

print(foobar)

print(test_data_female["AbsDoseTest"]-test_data_male["AbsDoseTest"])
print(test_data_female["AbsDoseTest"]-test_data_male["AbsDoseTest"])

print(ss.pearsonr(test_data_male["AbsDoseTest"], test_data_male["PBN Neut"]))
print(ss.pearsonr(test_data_male["AbsDoseTest"], test_data_male["PBN Neut"]))
print(ss.pearsonr(foobar["Dose"], foobar["PBN Neut"]))

          MBqHr        Mass        PBN   PBN Neut Sex  AbsDoseTest
0   1244.878159   49.740000  11.382114  15.686275   F     2.027505
1    812.285213   77.970000  40.723982  40.000000   M     0.843960
2    762.123639   54.821491  19.607843  38.571429   M     1.126200
3   1268.761005   70.000000  31.141869  26.562500   M     1.468327
4    922.468952   45.940697  31.983806  43.478261   F     1.626654
5   2682.878504   67.539589   3.921569   1.200000   M     3.217981
6   1433.589559   49.324036   8.695652  11.904762   F     2.354546
7   2778.150999  100.381851  37.719298  28.000000   M     2.242031
8   1076.224512   33.540511   9.693878  17.647059   F     2.599406
9   1017.625260   60.300350  12.371134  13.953488   F     1.367127
10  2220.559492   86.719036  20.647773   9.876543   M     2.074383
11   825.667711   76.148987  46.907216  42.105263   M     0.878379
12   963.698540   34.642697  24.899598  27.272727   F     2.253567
13  1391.411185   51.848888  24.509804  25.641026   M     2.17

In [135]:
print((0.113-0.1012)/0.1012)

0.11660079051383404


In [2]:
energies = pd.read_excel("utils/decdataenergies.xlsx", sheet_name="AllElectrons")
saf_values = pd.read_excel("utils/saf_vals_male_table.xlsx")

energy_look_up_vector = saf_values["Energy"].values

print(energy_look_up_vector)

print(saf_values)



[1.0e-03 1.5e-03 2.0e-03 3.0e-03 4.0e-03 5.0e-03 6.0e-03 8.0e-03 1.0e-02
 1.5e-02 2.0e-02 3.0e-02 4.0e-02 5.0e-02 6.0e-02 8.0e-02 1.0e-01 1.5e-01
 2.0e-01 3.0e-01 4.0e-01 5.0e-01 6.0e-01 8.0e-01 1.0e+00 1.5e+00 2.0e+00
 3.0e+00 4.0e+00 5.0e+00 6.0e+00 8.0e+00 1.0e+01]
     Energy     100CF      90CF      80CF      70CF      60CF      50CF  \
0    0.0010  1.000543  0.999679  0.999246  0.999895  0.999679  0.999462   
1    0.0015  1.000543  0.999679  0.999246  0.998382  0.997085  0.997302   
2    0.0020  1.000543  0.999679  0.997518  0.996869  0.995789  0.995140   
3    0.0030  1.000543  0.997734  0.995789  0.995357  0.993196  0.991899   
4    0.0040  0.998382  0.997734  0.995789  0.993844  0.991899  0.990818   
5    0.0050  0.998382  0.997734  0.995789  0.992331  0.990602  0.988657   
6    0.0060  0.998382  0.997734  0.994060  0.992331  0.989306  0.987577   
7    0.0080  0.998382  0.995789  0.994060  0.990819  0.988009  0.985416   
8    0.0100  0.998382  0.995789  0.992331  0.989306  0.9

In [3]:
# Set up a table for CF-100, using all the
# electron energies from DECDATA

len_of_table = len(energies)
print(len_of_table)

cf_factors = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.7])

energy_absorbtion_frame = pd.DataFrame(columns=["Energy", "Y", "phi"], 
                                       index=np.arange(len_of_table))

energy_absorbtion_frame["Energy"] = energies["E (MeV)"]
energy_absorbtion_frame["Y"] = energies["Y (/nt)"]

for cf in cf_factors:

    CF_string = (str(int(cf*100))+"CF")

    for i in range(len_of_table):

        energy_to_look_up = (energy_absorbtion_frame.at[i, "Energy"])

        index = (np.abs(energy_look_up_vector-energy_to_look_up)).argmin()

        phi = (saf_values.at[index, CF_string])

        energy_absorbtion_frame.at[i, "phi"] = phi
        #energy_absorbtion_frame.at[i, "phi"] = 1.0

    s_factor = (np.sum(energy_absorbtion_frame["Energy"]*energy_absorbtion_frame["Y"]*energy_absorbtion_frame["phi"]))

    #print((s_factor))
    print(s_factor*1.602e-13)

55
4.512637955978169e-15
6.4984481622993395e-15
8.455631090453207e-15
1.041439383670304e-14
1.2349598172452836e-14
1.4301661864726334e-14
1.6224589150007205e-14
1.8116494335265856e-14
2.002000514033847e-14
2.192569598323203e-14
1.6224589150007205e-14


In [4]:
energy_per_desintegration = 4.5126e-15

s_factor = energy_per_desintegration

print(s_factor)

mass_am = (1.03*((1-0.1025)*0.1)/1e3)
#mass_am = 50*(1-0.135)*0.7*1.03*1e-3


Dose_1_MBqHrs = 3600*1e6*s_factor/mass_am

print(Dose_1_MBqHrs, " Factor")

old_factor = 0.0724/(1-0.135)

print(100*(Dose_1_MBqHrs-old_factor)/old_factor, " Diff factor")

#print((Dose_1_MBqHrs-0.0724)/0.0724)

#energy_per_MBqhrs = energy_per_desintegration*3600*1e6

#print(energy_per_MBqhrs)



4.5126e-15
0.17573475403629282  Factor
109.95934011242164  Diff factor


In [5]:
print(0.0905*(1-0.135))

0.07828249999999999
